In [1]:
from gensim import corpora, models
#jieba.posseg是分词器  ”我爱北京天安门“会变成我 r    爱 v    北京 ns    天安门 ns
import jieba.posseg as jp, jieba
import reader
import numpy as np
# 文本集
# file_name = 'select_description_from_sys_district.csv'
file_name = 'hotel_description_small.csv'
if reader.check_file(file_name, url=''):
    hotelsDesc = reader.read_data_as_array(file_name)
else:
    quit()
# print(hotelsDesc)
# texts = [
#     '美国教练坦言，没输给中国女排，是输给了郎平',
#     '美国无缘四强，听听主教练的评价',
#     '中国女排晋级世锦赛四强，全面解析主教练郎平的执教艺术',
#     '为什么越来越多的人买MPV，而放弃SUV？跑一趟长途就知道了',
#     '跑了长途才知道，SUV和轿车之间的差距',
#     '家用的轿车买什么好']
# 分词过滤条件
# jieba.add_word('一方天地', 9, 'n')
# n	普通名词	f	方位名词	s	处所名词	t	时间名词
# nr	人名	ns	地名	nt	机构团体名	nw	作品名
# nz	其他专名	v	普通动词	vd	动副词	vn	名动词
# a	形容词	ad	副形词	an	名形词	d	副词
# m	数量词	q	量词	r	代词	p	介词
# c	连词	u	助词	xc	其他虚词	w	
# PER	人名	LOC	地名	ORG	机构名	TIME	时间
flags = ('n', 'f', 's', 't', 'nr', 'ns', 'nt', 'nw', 'nz','v','vd','vn','a','ad','an','d','q','r','p','c','u','xc','PER','LOC','ORG')  # 词性
stop_file_name = 'stopwords.txt'
if reader.check_file(stop_file_name, url=''):
    stopWords = reader.read_data_as_array(stop_file_name)
else:
    quit()

stopword_list = set(stopWords)  # 停词。格式为{'', '她', '无', '至', '前后', '即令', '大', '往', '不特', '有的', '替代',。。。}


文件  hotel_description_small.csv 存在。盛宴大幕拉开了。
段落处理完毕
文件  stopwords.txt 存在。盛宴大幕拉开了。
段落处理完毕


In [2]:
training_size = 1000
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'

In [3]:
sentences = []
labels = []
training_sentences = sentences[:training_size]
training_labels = labels[:training_size]
testing_sentences = sentences[training_size:]
testing_labels = labels[training_size:]

In [4]:
# 分词
words_ls = []
words_speech = []
# 将texts每个item的句子拆成单词
# jieba.enable_paddle() #启动paddle模式
whole_words = []
for hotelDesc in hotelsDesc:
    temp_words = [w.word for w in jp.cut(hotelDesc,use_paddle=True) if w.flag in flags and w.word not in stopWords and len(w.word) > 0]
    temp_speechs = [w.flag for w in jp.cut(hotelDesc,use_paddle=True) if w.flag in flags and w.word not in stopWords and len(w.word) > 0]
#     words = [w.word for w in jp.cut(text) if w.flag in flags and w.word not in stopwords and len(w.word) > 0]
#     print('---{}, {}',temp_words, len(temp_words))
#     print('---{}, {}',temp_speechs, len(temp_speechs))
    if len(temp_words) > 0:
        #[[句一分词列表],[句二分词列表],...[句五分词列表]]
        words_ls.append(temp_words)
        #[[句一词性列表],[句二词性列表],...[句五词性列表]]
        words_speech.append(temp_speechs)
        #五句合并的一个列表
        whole_words.extend(temp_words)
print('++++++++{}'.format(whole_words[:30]))
print('whole_words length:', len(whole_words))
print('remove duplicate whole_words length:', len(list(set(whole_words))))


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.579 seconds.
Prefix dict has been built successfully.


++++++++['开业', '间房', '上海', '时生隅', '园林', '文化', '酒店', '秋末', '时生隅', '初见', '心愿', '想要', '快节奏', '都市', '讲诉', '时光', '生命', '角落', '对错', '闻', '浮沉', '便是', '时生隅', '由来', '秉承', '初心', '川沙', '地铁站', '商圈', '觅得']
whole_words length: 451
remove duplicate whole_words length: 296


In [5]:
# for w in jp.cut(text,use_paddle=True):
#天安门 ns  其中天安门用w.word   ns用w.flag
#     print('---{}',w)
# 构造词典 这个词典是将拼接后的文本放进去，每项都是唯一的。试验下来可以去重
dictionary = corpora.Dictionary(words_ls)
#print(dictionary.token2id) #{'上海': 0, '交融': 1, '便是': 2, '充满': 3, '共': 4, '初心': 5, '初见': 6, '别样': 7}
print(dictionary.doc2idx)
#可以用dictionary.save('/tmp/deerwester.dict')保存字典
# for i in dictionary:
#     print('_+_+_+_+_+_+{}', i)
# 基于词典，使【词】→【稀疏向量】，并将向量放入列表，形成【稀疏向量集】
#这里就是创建每个句子的词组的词频
#这里可以直接从网上读取语料库
corpus = [dictionary.doc2bow(words) for words in words_ls]
# print(corpus)

<bound method Dictionary.doc2idx of <gensim.corpora.dictionary.Dictionary object at 0x7f8d5cf02668>>


In [6]:
# lda模型，num_topics设置主题的个数
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=10)
# 打印所有主题，每个主题显示10个词
for topic in lda.print_topics(num_words=10, num_topics=10):
    print(topic)
# 主题推断
# print(lda.inference(corpus))

(0, '0.051*"酒店" + 0.026*"间" + 0.025*"上海" + 0.021*"分钟" + 0.019*"车程" + 0.016*"迪士尼" + 0.016*"装修" + 0.015*"店" + 0.015*"维也纳" + 0.014*"机场"')
(1, '0.009*"酒店" + 0.007*"上海" + 0.006*"站" + 0.005*"约" + 0.005*"迪士尼" + 0.005*"享受" + 0.005*"服务" + 0.005*"号线" + 0.005*"地铁" + 0.004*"均"')
(2, '0.004*"酒店" + 0.004*"上海" + 0.004*"万信" + 0.004*"约" + 0.004*"服务" + 0.004*"分钟" + 0.004*"享受" + 0.004*"号线" + 0.004*"地铁" + 0.003*"世博园"')
(3, '0.046*"酒店" + 0.019*"上海" + 0.013*"号线" + 0.013*"约" + 0.011*"地铁" + 0.010*"位于" + 0.009*"万信" + 0.009*"新" + 0.009*"享受" + 0.009*"迪士尼"')
(4, '0.031*"酒店" + 0.019*"迪士尼" + 0.012*"站" + 0.012*"上海" + 0.012*"南" + 0.012*"周" + 0.011*"世界" + 0.010*"浦东" + 0.010*"直达" + 0.010*"周边"')
(5, '0.021*"酒店" + 0.013*"迪士尼" + 0.010*"上海" + 0.009*"站" + 0.008*"浦东" + 0.008*"设计师" + 0.007*"宽敞" + 0.007*"号线" + 0.007*"更" + 0.006*"乐园"')
(6, '0.020*"酒店" + 0.020*"万信" + 0.016*"上海" + 0.015*"约" + 0.015*"服务" + 0.011*"享受" + 0.011*"分钟" + 0.010*"号线" + 0.010*"会议" + 0.009*"地铁"')
(7, '0.023*"酒店" + 0.016*"上海" + 0.016*"迪士尼" + 0.012*"站" + 0.01

In [7]:
for e, values in enumerate(lda.inference(corpus)[0]):
    print(words_ls[e])
    for ee, value in enumerate(values):
        print('\t主题%d推断值%.2f' % (ee, value))

['开业', '间房', '上海', '时生隅', '园林', '文化', '酒店', '秋末', '时生隅', '初见', '心愿', '想要', '快节奏', '都市', '讲诉', '时光', '生命', '角落', '对错', '闻', '浮沉', '便是', '时生隅', '由来', '秉承', '初心', '川沙', '地铁站', '商圈', '觅得', '天地', '充满', '江南', '气息', '园林', '叶园', '长廊', '湖景', '古雅', '庭院', '大气', '快节奏', '民俗文化', '交融', '幽静', '明快', '小憩', '居', '别样', '天地', '时生隅', '新老朋友', '叶园', '期待', '共', '当下', '时光', '言欢']
	主题0推断值0.10
	主题1推断值0.10
	主题2推断值0.10
	主题3推断值0.10
	主题4推断值0.10
	主题5推断值0.10
	主题6推断值0.10
	主题7推断值0.10
	主题8推断值0.10
	主题9推断值58.10
['开业', '装修', '间', '房云', '夜泊', '酒店', '上海浦东国际机场', '店', '位于', '浦东新区', '祝桥秋亭', '路', '毗邻', '上海浦东国际机场', '车程', '分钟', '上海', '迪士尼', '分钟', '车程', '上海', '动物园', '分钟', '车程', '酒店', '地理位置', '优越', '配备', '大巴', '中巴', '豪华轿车', '接送', '客户', '酒店', '拥有', '房间', '床', '间', '行政', '大床', '间', '标间', '间亲', '子房', '间']
	主题0推断值45.10
	主题1推断值0.10
	主题2推断值0.10
	主题3推断值0.10
	主题4推断值0.10
	主题5推断值0.10
	主题6推断值0.10
	主题7推断值0.10
	主题8推断值0.10
	主题9推断值0.10
['开业', '装修', '间房', '维也纳', '酒店', '上海浦东', '机场', '店', '位于', '浦东新区', '川南', '奉', '公路', '近晨', '阳路', '迪士尼', '直线', '距离', '约

In [8]:
text5 = '2013年开业74间房。海友酒店（上海德平路地铁站店）位于浦东新区德平路，毗邻上海八佰伴、陆家嘴、世纪公园、金桥国际商业中心。酒店附近多部公交与地铁可通往人民广场、外滩、南京路步行街等繁华商业区及旅游景点，交通便捷。 　　海友酒店（上海德平路地铁站店）拥有大床房、家庭房、双床房等多种房型供您选择，可满足顾客不同的入住需求。客房内配有24小时热水、液晶电视机等完善的配套设施，温馨舒适。 　　海友客栈（上海德平路店）竭诚为您提供至善至美的上佳服务，欢迎您随时光临下榻。'
# bow = dictionary.doc2bow([word.word for word in jp.cut(text5) if word.flag in flags and word.word not in stopwords])
bow = dictionary.doc2bow([w.word for w in jp.cut(text5,use_paddle=True) if w.flag in flags and w.word not in stopWords and len(w.word) > 0])
ndarray = lda.inference([bow])[0]
print(text5)
for e, value in enumerate(ndarray[0]):
    print('\t主题%d推断值%.2f' % (e, value))

2013年开业74间房。海友酒店（上海德平路地铁站店）位于浦东新区德平路，毗邻上海八佰伴、陆家嘴、世纪公园、金桥国际商业中心。酒店附近多部公交与地铁可通往人民广场、外滩、南京路步行街等繁华商业区及旅游景点，交通便捷。 　　海友酒店（上海德平路地铁站店）拥有大床房、家庭房、双床房等多种房型供您选择，可满足顾客不同的入住需求。客房内配有24小时热水、液晶电视机等完善的配套设施，温馨舒适。 　　海友客栈（上海德平路店）竭诚为您提供至善至美的上佳服务，欢迎您随时光临下榻。
	主题0推断值23.78
	主题1推断值0.10
	主题2推断值0.10
	主题3推断值10.19
	主题4推断值0.10
	主题5推断值0.10
	主题6推断值0.10
	主题7推断值0.10
	主题8推断值0.10
	主题9推断值2.32


In [9]:
word_id = dictionary.doc2idx(['开业'])[0]
print(word_id)
for i in lda.get_term_topics(word_id):
    print('【开业】与【主题{}】的关系值:{}'.format((i[0], i[1]*100)))

21


In [10]:
for word, word_id in dictionary.token2id.items():
    print(word, lda.get_term_topics(word_id, minimum_probability=1e-8))

上海 [(0, 0.022291364), (1, 0.00041396776), (2, 2.352925e-06), (3, 0.015342115), (4, 0.0065355925), (5, 0.002589161), (6, 0.009588584), (7, 0.009966596), (8, 0.0027164428), (9, 0.007513369)]
交融 [(0, 1.863801e-07), (1, 8.6435625e-07), (2, 1.0109657e-06), (3, 2.3266162e-07), (4, 4.0074963e-07), (5, 1.2997954e-06), (6, 4.3899797e-07), (7, 0.00026112405), (8, 0.00020810525), (9, 0.005108757)]
便是 [(0, 1.8637868e-07), (1, 8.643612e-07), (2, 1.0109464e-06), (3, 2.3266406e-07), (4, 4.007374e-07), (5, 1.2895622e-06), (6, 4.3899377e-07), (7, 0.0002487342), (8, 0.00019969742), (9, 0.0051736548)]
充满 [(0, 1.8637655e-07), (1, 8.6435296e-07), (2, 1.0109464e-06), (3, 2.3266362e-07), (4, 4.0074275e-07), (5, 1.1000093e-06), (6, 4.3900133e-07), (7, 0.0001350146), (8, 0.0004747033), (9, 0.004951518)]
共 [(0, 1.8637833e-07), (1, 8.643447e-07), (2, 1.0109407e-06), (3, 2.3265896e-07), (4, 4.0074582e-07), (5, 1.3383046e-06), (6, 4.3899712e-07), (7, 0.00016863037), (8, 0.00034260956), (9, 0.0050647114)]
初心 [(0, 1

In [11]:
print(lda.show_topic(0, 9999))
print('概率总和', sum(i[1] for i in lda.show_topic(0, 9999)))

[('酒店', 0.051265478), ('间', 0.025630092), ('上海', 0.025274957), ('分钟', 0.021430967), ('车程', 0.0193788), ('迪士尼', 0.016059235), ('装修', 0.01604091), ('店', 0.014890522), ('维也纳', 0.014531012), ('机场', 0.013851668), ('上海浦东国际机场', 0.013127736), ('浦东新区', 0.012636036), ('开业', 0.012225713), ('位于', 0.011365958), ('动物园', 0.011142198), ('上海浦东', 0.009514815), ('休闲', 0.009233297), ('宽敞', 0.009002712), ('约', 0.008644382), ('拥有', 0.008415594), ('地铁', 0.008402709), ('服务', 0.008100696), ('配备', 0.008062878), ('号线', 0.007149182), ('提供', 0.0069819055), ('早餐', 0.0069048554), ('祝桥秋亭', 0.0068765), ('房云', 0.0068764733), ('房间', 0.006876471), ('中巴', 0.0068764635), ('路', 0.006876459), ('大巴', 0.0068764533), ('接送', 0.00687645), ('客户', 0.006876445), ('行政', 0.0068764323), ('毗邻', 0.006876427), ('间亲', 0.0068764235), ('子房', 0.006876409), ('标间', 0.006876403), ('床', 0.0068764025), ('豪华轿车', 0.0068763914), ('优越', 0.0068763467), ('大床', 0.0068762945), ('地理位置', 0.006876246), ('夜泊', 0.006876222), ('客房', 0.0066334293), ('时尚', 0.0063

In [12]:
vocab = dictionary.token2id
print(vocab)
print ('{} unique characters'.format(len(vocab)))

{'上海': 0, '交融': 1, '便是': 2, '充满': 3, '共': 4, '初心': 5, '初见': 6, '别样': 7, '古雅': 8, '叶园': 9, '商圈': 10, '园林': 11, '地铁站': 12, '大气': 13, '天地': 14, '对错': 15, '小憩': 16, '居': 17, '川沙': 18, '幽静': 19, '庭院': 20, '开业': 21, '当下': 22, '心愿': 23, '快节奏': 24, '想要': 25, '文化': 26, '新老朋友': 27, '时光': 28, '时生隅': 29, '明快': 30, '期待': 31, '民俗文化': 32, '气息': 33, '江南': 34, '浮沉': 35, '湖景': 36, '生命': 37, '由来': 38, '秉承': 39, '秋末': 40, '觅得': 41, '角落': 42, '言欢': 43, '讲诉': 44, '都市': 45, '酒店': 46, '长廊': 47, '间房': 48, '闻': 49, '上海浦东国际机场': 50, '中巴': 51, '优越': 52, '位于': 53, '分钟': 54, '动物园': 55, '地理位置': 56, '夜泊': 57, '大巴': 58, '大床': 59, '子房': 60, '客户': 61, '床': 62, '店': 63, '房云': 64, '房间': 65, '拥有': 66, '接送': 67, '标间': 68, '毗邻': 69, '浦东新区': 70, '祝桥秋亭': 71, '行政': 72, '装修': 73, '豪华轿车': 74, '路': 75, '车程': 76, '迪士尼': 77, '配备': 78, '间': 79, '间亲': 80, '上海浦东': 81, '中心': 82, '交通': 83, '人性化': 84, '休闲': 85, '会务': 86, '会议室': 87, '体现': 88, '便利': 89, '便捷': 90, '停车场': 91, '公路': 92, '公里': 93, '农艺': 94, '凌空': 95, '到达': 96, '号线': 97, '周围': 98

In [13]:
import numpy as np

char2idx = {u:i for i, u in enumerate(vocab)}
# print(char2idx)
idx2char = {u:i for u, i in enumerate(vocab)}
print(char2idx)

{'上海': 0, '交融': 1, '便是': 2, '充满': 3, '共': 4, '初心': 5, '初见': 6, '别样': 7, '古雅': 8, '叶园': 9, '商圈': 10, '园林': 11, '地铁站': 12, '大气': 13, '天地': 14, '对错': 15, '小憩': 16, '居': 17, '川沙': 18, '幽静': 19, '庭院': 20, '开业': 21, '当下': 22, '心愿': 23, '快节奏': 24, '想要': 25, '文化': 26, '新老朋友': 27, '时光': 28, '时生隅': 29, '明快': 30, '期待': 31, '民俗文化': 32, '气息': 33, '江南': 34, '浮沉': 35, '湖景': 36, '生命': 37, '由来': 38, '秉承': 39, '秋末': 40, '觅得': 41, '角落': 42, '言欢': 43, '讲诉': 44, '都市': 45, '酒店': 46, '长廊': 47, '间房': 48, '闻': 49, '上海浦东国际机场': 50, '中巴': 51, '优越': 52, '位于': 53, '分钟': 54, '动物园': 55, '地理位置': 56, '夜泊': 57, '大巴': 58, '大床': 59, '子房': 60, '客户': 61, '床': 62, '店': 63, '房云': 64, '房间': 65, '拥有': 66, '接送': 67, '标间': 68, '毗邻': 69, '浦东新区': 70, '祝桥秋亭': 71, '行政': 72, '装修': 73, '豪华轿车': 74, '路': 75, '车程': 76, '迪士尼': 77, '配备': 78, '间': 79, '间亲': 80, '上海浦东': 81, '中心': 82, '交通': 83, '人性化': 84, '休闲': 85, '会务': 86, '会议室': 87, '体现': 88, '便利': 89, '便捷': 90, '停车场': 91, '公路': 92, '公里': 93, '农艺': 94, '凌空': 95, '到达': 96, '号线': 97, '周围': 98

In [14]:
import tensorflow as tf
# 设定每个输入句子长度的最大值
seq_length = 60
examples_per_epoch = len(whole_words)//seq_length
text_as_int = np.array([char2idx[c] for c in whole_words])
print(text_as_int)
# 创建训练样本 / 目标
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

[ 21  48   0  29  11  26  46  40  29   6  23  25  24  45  44  28  37  42
  15  49  35   2  29  38  39   5  18  12  10  41  14   3  34  33  11   9
  47  36   8  20  13  24  32   1  19  30  16  17   7  14  29  27   9  31
   4  22  28  43  21  73  79  64  57  46  50  63  53  70  71  75  69  50
  76  54   0  77  54  76   0  55  54  76  46  56  52  78  58  51  74  67
  61  46  66  65  62  79  72  59  79  68  80  60  79  21  73  48 131  46
  81 120  63  53  70 107 104  92 139 142  77 127 137 130  93  90  96 101
  97  95 138  83  89  46  98 126 134 147 114 136   0 113 100  82  77   0
  55  94 103 123 115 131  46  81 120  63 131 116 141  46  73 135 132 112
 145 146 105 122 118 109 108 129 102  88  84 124  46 106  91  85 133 106
  87 140 111 128 117 121 120 110 143 144 119  99  85  86 125  46  21  48
   0 148  46  53  70 178 173 161 214 213 165 207 101  97 163 185 190 101
  97 199 138  78 150  91 195 113 100  82 130  54 175 151 164 167 183   0
 194   0 208 130  54  46  66 105 193 211 132 188  4

In [15]:
words = sorted(list(set(whole_words)))
word_indices = dict((word, words.index(word)) for word in words)
print(words)
print(word_indices)

maxlen = 30
sentences = []
next_word = []

for i in range(0, len(whole_words) - maxlen):
    sentences.append(whole_words[i: i + maxlen])
    next_word.append(whole_words[i + maxlen])
print('提取的句子总数:', len(sentences))

x = np.zeros((len(sentences), maxlen), dtype='float32') # Embedding的输入是2维张量（句子数，序列长度）
y = np.zeros((len(sentences)), dtype='float32')
for i, sentence in enumerate(sentences):
    for t, word in enumerate(sentence):
        x[i, t] = word_indices[word]
    y[i] = word_indices[next_word[i]]

['万信', '万达', '三诺', '上海', '上海浦东', '上海浦东国际机场', '专业', '专属', '世博园', '世界', '中巴', '中心', '主打', '丽都', '乐园', '交融', '交通', '享受', '人性化', '以下', '企业级', '休闲', '优质', '优越', '会务', '会议', '会议室', '位于', '体现', '体验', '佳肴', '便利', '便捷', '便是', '保证', '信厅', '停车场', '充满', '先进', '免费', '公路', '公里', '共', '典雅', '内环线', '农艺', '冰雪', '凌空', '分钟', '初心', '初见', '别样', '到达', '前台', '加工', '动物园', '北洋', '区', '区内', '华丽', '南', '南京路', '厅', '口味', '古雅', '可达', '叶园', '号线', '各类', '名厨', '周', '周围', '周浦镇', '周边', '咨询', '品牌', '商务', '商圈', '商城', '园林', '国际博览', '地处', '地理位置', '地铁', '地铁站', '均', '声学', '处处', '外滩', '夜泊', '大巴', '大床', '大气', '大观园', '天地', '奉', '娱乐', '婚礼', '子房', '定时', '客户', '客房', '宴会厅', '宽敞', '对错', '尊贵', '小憩', '居', '山路', '川南', '川沙', '川菜', '布置', '带来', '带给', '年轻', '幽静', '广场', '床', '店', '店家', '庭院', '开业', '弄', '当下', '当天', '心愿', '快节奏', '想要', '房', '房云', '房间', '打造', '拥有', '接机', '接送', '控系统', '提供', '提前', '支持', '整体', '文化', '新', '新老朋友', '旅游', '旅游区', '旗下', '早餐', '时光', '时尚', '时生隅', '明快', '景点', '智能', '更', '服务', '期待', '机场', '极速', '标间', '横桥', '步行街', '每个', '毗邻'

In [16]:
import sys
print(np.round((sys.getsizeof(x) / 1024 / 1024 / 1024), 4), "GB") 
print(x.shape, y.shape)

0.0 GB
(421, 30) (421,)


In [17]:
# 批大小
BATCH_SIZE = 64

# 设定缓冲区大小，以重新排列数据集
# （TF 数据被设计为可以处理可能是无限的序列，
# 所以它不会试图在内存中重新排列整个序列。相反，
# 它维持一个缓冲区，在缓冲区重新排列元素。） 
BUFFER_SIZE = 10000

# dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# dataset

In [18]:
# 词集的长度
vocab_size = len(words)

# 嵌入的维度
embedding_dim = 256

# RNN 的单元数量
rnn_units = 1024
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [19]:
import tensorflow as tf
import os
# main_input = tf.keras.layers.Input(shape=(maxlen, ), dtype='float32') 
# model_1 = tf.keras.layers.Embedding(len(words), maxlen, input_length=maxlen)(main_input)
# model_1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(maxlen, return_sequences=True))(model_1)
# model_1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(maxlen))(model_1)
# output = tf.keras.layers.Dense(len(words), activation='softmax')(model_1)  
# model = tf.keras.models.Model(main_input, output)

model = build_model(
  vocab_size = len(words),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

# optimizer = tf.keras.optimizers.RMSprop(lr=3e-3)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

# 检查点保存至的目录
checkpoint_dir = './ldatraining_checkpoints'

# 检查点的文件名
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)
# model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
history = model.fit(x, y, epochs=10, batch_size=BATCH_SIZE, callbacks=[checkpoint_callback])
model.summary()

Train on 421 samples
Epoch 1/10
 64/421 [===>..........................] - ETA: 9s

InvalidArgumentError:  assertion failed: [Condition x == y did not hold element-wise:] [x (loss/dense_loss/SparseSoftmaxCrossEntropyWithLogits/Shape_1:0) = ] [1920 1] [y (loss/dense_loss/SparseSoftmaxCrossEntropyWithLogits/strided_slice:0) = ] [64 30]
	 [[node loss/dense_loss/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert (defined at <ipython-input-19-ca363faac450>:29) ]] [Op:__inference_distributed_function_2601]

Function call stack:
distributed_function


In [ ]:
def generate_text(model, start_string):
    num_generate = 100
    input_eval = [char2idx[s.word] for s in jp.cut(start_string)]
    input_eval = tf.expand_dims(input_eval, 0)
    # 空字符串用于存储结果
    text_generated = []

    # 低温度会生成更可预测的文本
    # 较高温度会生成更令人惊讶的文本
    # 可以通过试验以找到最好的设定
    temperature = 1.0

    # 这里批大小为 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        print(predictions)
        # 删除批次的维度
        predictions = tf.squeeze(predictions, 0)
        print(predictions)
        # 用分类分布预测模型返回的字符
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # 把预测字符和前面的隐藏状态一起传递给模型作为下一个输入
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"酒店服务"))